# 🚀 SwingAI Training Notebook V2

**AI-Powered Swing Trading for Indian Markets**

---

## 📋 Overview

This notebook trains the SwingAI ensemble model:
- **CatBoost** (35%) - Gradient boosting for tabular data
- **TFT** (35%) - Temporal Fusion Transformer for sequences
- **Stockformer** (30%) - STL-inspired transformer

## 🎯 Features
- **40 Pure OHLCV Features** (Price Action, SMC/ICT, Volume, MTF)
- **No External Dependencies** - Just stock price data
- **Rule-Based Filters** - VIX, FII/DII applied separately

## ⚙️ Requirements
- Google Colab with GPU (T4 recommended)
- ~2 hours training time
- ~5GB RAM

---

## 📦 How to Use

1. Open in Google Colab
2. Enable GPU: Runtime → Change runtime type → T4 GPU
3. Run all cells in order
4. Models will be saved to Google Drive

## 🔍 What I fixed (line-by-line level)

### 1) Dependencies (Step 1)
- Removed the hard pin on `yfinance==0.2.36` (often breaks in Colab due to Yahoo changes).
- Added `curl_cffi` (used by modern yfinance internals for more reliable fetching).
- Kept PyTorch + CatBoost + sklearn.

### 2) Yahoo Finance downloader (Step 3)
Your error:
- `Expecting value: line 1 column 1` + `No timezone found`

Root cause:
- Yahoo returns empty/blocked responses when many requests hit quickly (especially with threads in Colab).

Fix:
- Download in **small chunks**
- **threads=False**
- **retries** with exponential backoff

### 3) Notebook correctness
- Ensured the feature engineering cell is a **code cell** (so it actually runs).
- Ensured the “Training Complete” cell is **markdown** (not executed as Python).

If you get a new error later in training (CatBoost / PyTorch / CUDA OOM), paste it and I’ll patch the notebook again.


In [1]:
# ==============================
# Colab-stable dependencies
# ==============================

# 1) Clean conflicting packages (quiet but forceful)
%pip -q uninstall -y numpy pandas scipy scikit-learn catboost yfinance ta curl_cffi

# 2) Install a known-stable stack for Colab (CUDA is preinstalled by Colab for torch)
#    - Pin numpy < 2.0 to avoid incompatibilities in some libraries
#    - Pin pandas/scipy to stable versions
#    - Install yfinance + curl_cffi (important for Yahoo throttling issues)
%pip -q install "numpy<2.0" "pandas==2.2.2" "scipy<1.13" "scikit-learn==1.5.2" \
    "catboost==1.2.7" "yfinance>=0.2.54" "curl_cffi>=0.7.4" "ta>=0.11.0"

print("✅ Install done. Now go to Runtime → Restart runtime, then run the next cell.")


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires scipy>=1.13, but you have scipy 1.12.0 which is incompatible.
torchvision 0.24.0+cu126 requires torch==2.9.0, but you have torch 2.9.1 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7

In [2]:
import numpy as np, pandas as pd, scipy
import sklearn
import torch
import yfinance as yf
import catboost
import ta

print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("scipy:", scipy.__version__)
print("sklearn:", sklearn.__version__)
print("catboost:", catboost.__version__)
print("yfinance:", yf.__version__)
print("torch:", torch.__version__)
print("✅ GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("   Device:", torch.cuda.get_device_name(0))


import torch
print(f"✅ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   Device: {torch.cuda.get_device_name(0)}")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


numpy: 1.26.4
pandas: 2.2.2
scipy: 1.12.0
sklearn: 1.5.2
catboost: 1.2.7
yfinance: 1.0
torch: 2.9.1+cu128
✅ GPU Available: True
   Device: Tesla T4
✅ GPU Available: True
   Device: Tesla T4


## 📊 Step 2: Imports & Configuration

In [3]:

import os
import json
import warnings
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass
import yfinance as yf
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings('ignore')

# Configuration
@dataclass
class Config:
    LOOKBACK_DAYS: int = 60
    PREDICTION_HORIZON: int = 5
    MIN_HISTORY_DAYS: int = 252
    UP_THRESHOLD: float = 0.03
    DOWN_THRESHOLD: float = -0.02
    TRAIN_END: str = "2024-06-30"
    VAL_END: str = "2024-09-30"
    CATBOOST_WEIGHT: float = 0.35
    TFT_WEIGHT: float = 0.35
    STOCKFORMER_WEIGHT: float = 0.30
    BATCH_SIZE: int = 64
    LEARNING_RATE: float = 1e-3
    MAX_EPOCHS: int = 50
    PATIENCE: int = 5
    CATBOOST_ITERATIONS: int = 1000
    CATBOOST_DEPTH: int = 6
    CATBOOST_LR: float = 0.05
    TFT_HIDDEN_SIZE: int = 64
    TFT_ATTENTION_HEADS: int = 4
    TFT_DROPOUT: float = 0.1
    STOCKFORMER_D_MODEL: int = 64
    STOCKFORMER_N_HEADS: int = 4
    STOCKFORMER_N_LAYERS: int = 2
    NUM_FEATURES: int = 40
    MODEL_SAVE_PATH: str = "/content/drive/MyDrive/SwingAI/models/"

config = Config()

# 40 Features
FEATURE_NAMES = [
    'return_1d', 'return_5d', 'return_10d', 'return_20d', 'volatility_20d',
    'close_to_sma_20', 'close_to_sma_50', 'rsi_14_norm', 'macd_histogram_norm', 'bb_position',
    'structure_score', 'range_position', 'dist_to_swing_high', 'dist_to_swing_low',
    'in_discount', 'in_deep_discount', 'in_premium', 'near_bullish_ob', 'near_bearish_ob',
    'bullish_fvg', 'bearish_fvg', 'sweep_high', 'sweep_low', 'bos_bullish', 'bos_bearish',
    'volume_ratio', 'volume_trend', 'obv_slope', 'close_to_vwap',
    'buying_pressure', 'accumulation_score', 'big_volume_day', 'higher_high',
    'daily_trend', 'weekly_trend', 'monthly_trend', 'mtf_alignment',
    'weekly_range_pos', 'monthly_range_pos', 'trend_strength',
]

print(f"✅ Config loaded: {len(FEATURE_NAMES)} features")

✅ Config loaded: 40 features


## 📈 Step 3: Stock Universe & Data Download

### ✅ Colab Yahoo Finance download fix (important)

If you see errors like **`Expecting value: line 1 column 1`** or **`No timezone found`**, it usually means Yahoo returned an empty/blocked response (rate-limit / throttle).  
This notebook uses a **chunked, retrying, threads-disabled downloader** to avoid getting blocked.

Key ideas:
- **No threads** (threads often triggers throttling in Colab)
- **Small chunks** of tickers per request
- **Retries** with exponential backoff + jitter


In [4]:
# F&O Stocks universe (edit this list if you want fewer/more tickers)
FO_STOCKS = [
    "RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "ICICIBANK.NS",
    "HINDUNILVR.NS", "SBIN.NS", "BHARTIARTL.NS", "KOTAKBANK.NS", "ITC.NS",
    "LT.NS", "AXISBANK.NS", "ASIANPAINT.NS", "MARUTI.NS", "HCLTECH.NS",
    "SUNPHARMA.NS", "TITAN.NS", "BAJFINANCE.NS", "ULTRACEMCO.NS", "NTPC.NS",
    "WIPRO.NS", "NESTLEIND.NS", "POWERGRID.NS", "M&M.NS", "TATAMOTORS.NS",
    "JSWSTEEL.NS", "ADANIENT.NS", "ADANIPORTS.NS", "TATASTEEL.NS", "ONGC.NS",
    "TECHM.NS", "HDFCLIFE.NS", "DIVISLAB.NS", "BAJAJFINSV.NS", "GRASIM.NS",
    "DRREDDY.NS", "CIPLA.NS", "BRITANNIA.NS", "EICHERMOT.NS", "APOLLOHOSP.NS",
    "COALINDIA.NS", "SBILIFE.NS", "BPCL.NS", "INDUSINDBK.NS", "TATACONSUM.NS",
    "HEROMOTOCO.NS", "HINDALCO.NS", "BAJAJ-AUTO.NS", "LTIM.NS", "SHRIRAMFIN.NS",
    "TRENT.NS", "POLYCAB.NS", "PERSISTENT.NS", "DIXON.NS", "TATAELXSI.NS",
    "ABB.NS", "SIEMENS.NS", "HAL.NS", "BEL.NS", "IRCTC.NS",
    "ZOMATO.NS", "COFORGE.NS", "MUTHOOTFIN.NS", "INDHOTEL.NS", "BANKBARODA.NS",
    "PNB.NS", "IDFCFIRSTB.NS", "FEDERALBNK.NS", "CHOLAFIN.NS", "VEDL.NS",
]

import time, random
import pandas as pd
import yfinance as yf

def yf_download_safe(
    tickers,
    start: str,
    end: str,
    interval: str = "1d",
    auto_adjust: bool = True,
    max_retries: int = 5,
    chunk_size: int = 5,
    base_sleep: float = 1.5,
):
    """Robust yfinance downloader for Google Colab.

    Why this exists:
    - In Colab, Yahoo frequently rate-limits / returns empty HTML which breaks JSON parsing.
    - yfinance then throws: 'Expecting value: line 1 column 1' and 'No timezone found'.

    What we do:
    - Download tickers in small chunks
    - Disable threads (VERY IMPORTANT for Colab)
    - Retry with exponential backoff + jitter
    """
    tickers = [t.strip() for t in tickers if isinstance(t, str) and t.strip()]
    out = {}

    total = len(tickers)
    print(f"📥 Downloading {total} stocks (chunk={chunk_size}, retries={max_retries})...")

    for i in range(0, total, chunk_size):
        chunk = tickers[i:i+chunk_size]

        success = False
        last_err = None

        for attempt in range(1, max_retries + 1):
            try:
                df = yf.download(
                    tickers=" ".join(chunk),
                    start=start,
                    end=end,
                    interval=interval,
                    group_by="ticker",
                    auto_adjust=auto_adjust,
                    threads=False,     # 🔥 critical for Colab stability
                    progress=False,
                )

                # yfinance returns:
                # - MultiIndex columns when multiple tickers
                # - Single-level columns for a single ticker
                if isinstance(df.columns, pd.MultiIndex):
                    for t in chunk:
                        if t in df.columns.get_level_values(0):
                            tdf = df[t].dropna(how="all")
                            if not tdf.empty:
                                out[t] = tdf
                else:
                    # Single ticker case (chunk_size may be 1)
                    t = chunk[0]
                    tdf = df.dropna(how="all")
                    if not tdf.empty:
                        out[t] = tdf

                success = True
                break

            except Exception as e:
                last_err = e
                # backoff + jitter
                sleep_s = base_sleep * (2 ** (attempt - 1)) + random.random()
                time.sleep(sleep_s)

        # progress
        done = min(i + chunk_size, total)
        if done % 10 == 0 or done == total:
            print(f"   Progress: {done}/{total} | downloaded={len(out)}")

        if not success and last_err is not None:
            print(f"⚠️  Chunk failed: {chunk} | last error: {last_err}")

        # small delay between chunks to reduce throttling
        time.sleep(base_sleep + random.random())

    print(f"✅ Downloaded {len(out)} stocks")
    return out

# Download ~5 years of daily data
stock_data = yf_download_safe(FO_STOCKS, start="2019-01-01", end="2024-12-31", interval="1d")

# Optional: show which tickers failed (helps debugging)
failed = [t for t in FO_STOCKS if t not in stock_data]
print(f"❗ Failed tickers: {len(failed)}")
print(failed[:30])


📥 Downloading 70 stocks (chunk=5, retries=5)...
   Progress: 10/70 | downloaded=10
   Progress: 20/70 | downloaded=20


ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TATAMOTORS.NS"}}}
ERROR:yfinance:$TATAMOTORS.NS: possibly delisted; no timezone found
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['TATAMOTORS.NS']: possibly delisted; no timezone found


   Progress: 30/70 | downloaded=29
   Progress: 40/70 | downloaded=39
   Progress: 50/70 | downloaded=49
   Progress: 60/70 | downloaded=59


ERROR:yfinance:$ZOMATO.NS: possibly delisted; no timezone found
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ZOMATO.NS']: possibly delisted; no timezone found


   Progress: 70/70 | downloaded=68
✅ Downloaded 68 stocks
❗ Failed tickers: 2
['TATAMOTORS.NS', 'ZOMATO.NS']


#### Tip: If Yahoo blocks you (still)
If you still get many failed tickers, try:
- Reduce universe for a test run: `FO_STOCKS = FO_STOCKS[:10]`
- Increase `base_sleep` to 2.5–3.0 seconds
- Run again (Yahoo throttling is often temporary)

This notebook continues even if a few tickers fail — it will train on the downloaded set.


In [5]:
def calculate_features(df):
    """Calculate all 40 features from OHLCV data"""
    data = df.copy()

    # === PRICE ACTION (10) ===
    data['return_1d'] = data['Close'].pct_change(1)
    data['return_5d'] = data['Close'].pct_change(5)
    data['return_10d'] = data['Close'].pct_change(10)
    data['return_20d'] = data['Close'].pct_change(20)
    data['volatility_20d'] = data['return_1d'].rolling(20).std() * np.sqrt(252)

    data['sma_20'] = data['Close'].rolling(20).mean()
    data['sma_50'] = data['Close'].rolling(50).mean()
    data['close_to_sma_20'] = (data['Close'] - data['sma_20']) / data['sma_20']
    data['close_to_sma_50'] = (data['Close'] - data['sma_50']) / data['sma_50']

    # RSI
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    data['rsi_14_norm'] = (100 - (100 / (1 + rs)) - 50) / 50

    # MACD
    ema_12 = data['Close'].ewm(span=12).mean()
    ema_26 = data['Close'].ewm(span=26).mean()
    macd = ema_12 - ema_26
    macd_signal = macd.ewm(span=9).mean()
    data['macd_histogram_norm'] = (macd - macd_signal) / data['Close']

    # Bollinger Bands
    bb_mid = data['Close'].rolling(20).mean()
    bb_std = data['Close'].rolling(20).std()
    bb_upper = bb_mid + 2 * bb_std
    bb_lower = bb_mid - 2 * bb_std
    data['bb_position'] = (data['Close'] - bb_lower) / (bb_upper - bb_lower + 1e-10)

    # === SMC/ICT (15) ===
    data['swing_high'] = data['High'].rolling(10, center=True).max()
    data['swing_low'] = data['Low'].rolling(10, center=True).min()

    data['prev_swing_high'] = data['swing_high'].shift(10)
    data['prev_swing_low'] = data['swing_low'].shift(10)
    data['higher_high'] = (data['swing_high'] > data['prev_swing_high']).astype(int)
    data['higher_low'] = (data['swing_low'] > data['prev_swing_low']).astype(int)
    data['lower_high'] = (data['swing_high'] < data['prev_swing_high']).astype(int)
    data['lower_low'] = (data['swing_low'] < data['prev_swing_low']).astype(int)

    data['structure_score'] = (data['higher_high'].rolling(5).sum() +
                               data['higher_low'].rolling(5).sum() -
                               data['lower_high'].rolling(5).sum() -
                               data['lower_low'].rolling(5).sum()) / 10

    range_high = data['High'].rolling(50).max()
    range_low = data['Low'].rolling(50).min()
    data['range_position'] = (data['Close'] - range_low) / (range_high - range_low + 1e-10)

    data['dist_to_swing_high'] = (data['swing_high'] - data['Close']) / data['Close']
    data['dist_to_swing_low'] = (data['Close'] - data['swing_low']) / data['Close']

    data['in_discount'] = (data['range_position'] < 0.5).astype(int)
    data['in_deep_discount'] = (data['range_position'] < 0.3).astype(int)
    data['in_premium'] = (data['range_position'] > 0.7).astype(int)

    vol_threshold = data['return_1d'].rolling(20).std() * 2
    data['near_bullish_ob'] = (data['return_1d'] > vol_threshold).rolling(10).sum()
    data['near_bearish_ob'] = (data['return_1d'] < -vol_threshold).rolling(10).sum()

    data['gap_up'] = ((data['Low'] > data['High'].shift(1)) & (data['return_1d'] > 0.01)).astype(int)
    data['gap_down'] = ((data['High'] < data['Low'].shift(1)) & (data['return_1d'] < -0.01)).astype(int)
    data['bullish_fvg'] = data['gap_up'].rolling(5).sum()
    data['bearish_fvg'] = data['gap_down'].rolling(5).sum()

    data['sweep_high'] = ((data['High'] > data['swing_high'].shift(1)) &
                          (data['Close'] < data['swing_high'].shift(1))).astype(int)
    data['sweep_low'] = ((data['Low'] < data['swing_low'].shift(1)) &
                         (data['Close'] > data['swing_low'].shift(1))).astype(int)

    data['bos_bullish'] = ((data['Close'] > data['swing_high'].shift(1)) &
                           (data['higher_high'] == 1)).astype(int)
    data['bos_bearish'] = ((data['Close'] < data['swing_low'].shift(1)) &
                           (data['lower_low'] == 1)).astype(int)

    # === VOLUME (8) ===
    data['volume_ma_20'] = data['Volume'].rolling(20).mean()
    data['volume_ratio'] = data['Volume'] / (data['volume_ma_20'] + 1e-10)
    data['volume_trend'] = data['Volume'].rolling(5).mean() / (data['Volume'].rolling(20).mean() + 1e-10)

    obv = (np.sign(data['Close'].diff()) * data['Volume']).cumsum()
    data['obv_slope'] = obv.diff(5) / (obv.rolling(20).std() + 1e-10)

    data['vwap'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()
    data['close_to_vwap'] = (data['Close'] - data['vwap']) / (data['vwap'] + 1e-10)

    data['buying_pressure'] = (data['Close'] - data['Low']) / (data['High'] - data['Low'] + 1e-10)
    data['accumulation_score'] = (data['buying_pressure'] * data['volume_ratio']).rolling(5).mean()
    data['big_volume_day'] = (data['volume_ratio'] > 2).astype(int)

    # === MULTI-TIMEFRAME (7) ===
    data['daily_trend'] = (data['Close'] > data['sma_20']).astype(int)

    data['weekly_close'] = data['Close'].rolling(5).mean()
    data['weekly_high'] = data['High'].rolling(5).max()
    data['weekly_low'] = data['Low'].rolling(5).min()
    data['weekly_trend'] = (data['weekly_close'] > data['weekly_close'].shift(5)).astype(int)

    data['monthly_close'] = data['Close'].rolling(21).mean()
    data['monthly_trend'] = (data['monthly_close'] > data['monthly_close'].shift(21)).astype(int)

    data['mtf_alignment'] = (data['daily_trend'] + data['weekly_trend'] + data['monthly_trend']) / 3

    data['weekly_range_pos'] = (data['Close'] - data['weekly_low']) / (data['weekly_high'] - data['weekly_low'] + 1e-10)
    data['monthly_range_pos'] = (data['Close'] - data['Low'].rolling(21).min()) / \
                                (data['High'].rolling(21).max() - data['Low'].rolling(21).min() + 1e-10)

    data['trend_strength'] = abs(data['close_to_sma_20']) + abs(data['close_to_sma_50'])

    # Select and clean
    features_df = data[FEATURE_NAMES].copy()
    features_df = features_df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return features_df

def create_labels(df):
    """Create labels: 0=SHORT, 1=NEUTRAL, 2=LONG"""
    forward_return = df['Close'].shift(-config.PREDICTION_HORIZON) / df['Close'] - 1
    labels = pd.Series(1, index=df.index)
    labels[forward_return >= config.UP_THRESHOLD] = 2   # LONG
    labels[forward_return <= config.DOWN_THRESHOLD] = 0  # SHORT
    return labels

# Calculate features for all stocks
print("🔧 Calculating features...")
feature_data = {}
labels_data = {}

for symbol, data in stock_data.items():
    try:
        features = calculate_features(data)
        labels = create_labels(data)
        if len(features) > config.LOOKBACK_DAYS + config.PREDICTION_HORIZON:
            feature_data[symbol] = features
            labels_data[symbol] = labels
    except:
        pass

print(f"✅ Processed {len(feature_data)} stocks")

🔧 Calculating features...
✅ Processed 68 stocks


## 📦 Step 5: Prepare Datasets

In [6]:
class SwingDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'features': self.features[idx], 'labels': self.labels[idx]}

# Create sequences
print("📊 Creating sequences...")
all_sequences, all_labels, all_dates = [], [], []

for symbol in feature_data.keys():
    features = feature_data[symbol].values
    labels = labels_data[symbol].values
    dates = feature_data[symbol].index

    for i in range(config.LOOKBACK_DAYS, len(features) - config.PREDICTION_HORIZON):
        seq = features[i-config.LOOKBACK_DAYS:i]
        label = labels[i]
        if not np.isnan(label) and not np.any(np.isnan(seq)):
            all_sequences.append(seq)
            all_labels.append(int(label))
            all_dates.append(dates[i])

sequences = np.array(all_sequences)
labels = np.array(all_labels)
dates_arr = np.array(all_dates)

print(f"✅ Total sequences: {len(sequences):,}")
print(f"   Shape: {sequences.shape}")

# Split by time
train_end = pd.Timestamp(config.TRAIN_END)
val_end = pd.Timestamp(config.VAL_END)

train_mask = dates_arr <= train_end
val_mask = (dates_arr > train_end) & (dates_arr <= val_end)
test_mask = dates_arr > val_end

train_dataset = SwingDataset(sequences[train_mask], labels[train_mask])
val_dataset = SwingDataset(sequences[val_mask], labels[val_mask])
test_dataset = SwingDataset(sequences[test_mask], labels[test_mask])

print(f"\n📊 Dataset Splits:")
print(f"   Train: {len(train_dataset):,}")
print(f"   Val:   {len(val_dataset):,}")
print(f"   Test:  {len(test_dataset):,}")

📊 Creating sequences...
✅ Total sequences: 95,960
   Shape: (95960, 60, 40)

📊 Dataset Splits:
   Train: 87,800
   Val:   4,352
   Test:  3,808


## 🤖 Step 6: Train CatBoost Model

In [7]:
print("="*60)
print("🚀 TRAINING CATBOOST MODEL")
print("="*60)

# Prepare data (use last day features for CatBoost)
X_train = train_dataset.features[:, -1, :].numpy()
y_train = train_dataset.labels.numpy()
X_val = val_dataset.features[:, -1, :].numpy()
y_val = val_dataset.labels.numpy()

print(f"Training samples: {len(X_train):,}")
print(f"Features: {X_train.shape[1]}")

# Train CatBoost
catboost_model = CatBoostClassifier(
    iterations=config.CATBOOST_ITERATIONS,
    depth=config.CATBOOST_DEPTH,
    learning_rate=config.CATBOOST_LR,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=50,
    task_type='GPU' if torch.cuda.is_available() else 'CPU',
    class_weights={0: 1.2, 1: 0.8, 2: 1.0}
)

catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

cat_val_acc = (catboost_model.predict(X_val) == y_val).mean()
print(f"\n✅ CatBoost Val Accuracy: {cat_val_acc:.4f}")

# Feature importance
importance = dict(zip(FEATURE_NAMES, catboost_model.feature_importances_))
sorted_imp = sorted(importance.items(), key=lambda x: x[1], reverse=True)
print("\n📊 Top 10 Important Features:")
for i, (f, v) in enumerate(sorted_imp[:10]):
    print(f"   {i+1}. {f}: {v:.4f}")

🚀 TRAINING CATBOOST MODEL
Training samples: 87,800
Features: 40
0:	learn: 0.5176687	test: 0.5393447	best: 0.5393447 (0)	total: 25.8ms	remaining: 25.8s
100:	learn: 0.5508259	test: 0.5688409	best: 0.5696328 (94)	total: 906ms	remaining: 8.06s
200:	learn: 0.5605168	test: 0.5704741	best: 0.5717114 (159)	total: 1.61s	remaining: 6.39s
bestTest = 0.5717113719
bestIteration = 159
Shrink model to first 160 iterations.

✅ CatBoost Val Accuracy: 0.4796

📊 Top 10 Important Features:
   1. dist_to_swing_high: 45.1263
   2. dist_to_swing_low: 18.3913
   3. weekly_range_pos: 14.2875
   4. return_5d: 4.8704
   5. volatility_20d: 2.8213
   6. close_to_vwap: 2.2493
   7. volume_trend: 1.7233
   8. macd_histogram_norm: 1.5717
   9. structure_score: 1.1442
   10. monthly_range_pos: 1.0915


## 🧠 Step 7: Train TFT Model

In [8]:
class TFTModel(nn.Module):
    def __init__(self):
        super().__init__()
        h, heads, drop = config.TFT_HIDDEN_SIZE, config.TFT_ATTENTION_HEADS, config.TFT_DROPOUT

        self.var_sel = nn.Sequential(nn.Linear(40, h), nn.ReLU(), nn.Dropout(drop), nn.Linear(h, 40), nn.Softmax(dim=-1))
        self.lstm = nn.LSTM(40, h, 2, batch_first=True, dropout=drop)
        self.attention = nn.MultiheadAttention(h, heads, dropout=drop, batch_first=True)
        self.grn = nn.Sequential(nn.Linear(h, h), nn.GELU(), nn.Dropout(drop), nn.Linear(h, h))
        self.grn_gate = nn.Sequential(nn.Linear(h, h), nn.Sigmoid())
        self.grn_norm = nn.LayerNorm(h)
        self.output = nn.Linear(h, 3)

    def forward(self, x):
        w = self.var_sel(x.mean(1))
        x = x * w.unsqueeze(1)
        o, _ = self.lstm(x)
        a, _ = self.attention(o, o, o)
        g = self.grn_gate(a) * self.grn(a)
        out = self.grn_norm(a + g)
        return self.output(out[:, -1, :])

print("="*60)
print("🚀 TRAINING TFT MODEL")
print("="*60)

tft_model = TFTModel().to(DEVICE)
train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE)
optimizer = torch.optim.AdamW(tft_model.parameters(), lr=config.LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

best_acc, patience_count, best_state = 0, 0, None

for epoch in range(config.MAX_EPOCHS):
    # Train
    tft_model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        out = tft_model(batch['features'].to(DEVICE))
        loss = criterion(out, batch['labels'].to(DEVICE))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(tft_model.parameters(), 1.0)
        optimizer.step()

    # Validate
    tft_model.eval()
    correct = sum((tft_model(b['features'].to(DEVICE)).argmax(1) == b['labels'].to(DEVICE)).sum().item()
                  for b in val_loader)
    acc = correct / len(val_dataset)
    print(f"Epoch {epoch+1}: Val Acc = {acc:.4f}")

    if acc > best_acc:
        best_acc, patience_count = acc, 0
        best_state = {k: v.cpu().clone() for k, v in tft_model.state_dict().items()}
    else:
        patience_count += 1
        if patience_count >= config.PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break

tft_model.load_state_dict(best_state)
print(f"\n✅ TFT Best Val Accuracy: {best_acc:.4f}")

🚀 TRAINING TFT MODEL
Epoch 1: Val Acc = 0.5602
Epoch 2: Val Acc = 0.5682
Epoch 3: Val Acc = 0.5579
Epoch 4: Val Acc = 0.5620
Epoch 5: Val Acc = 0.5699
Epoch 6: Val Acc = 0.5836
Epoch 7: Val Acc = 0.5715
Epoch 8: Val Acc = 0.6050
Epoch 9: Val Acc = 0.6089
Epoch 10: Val Acc = 0.6363
Epoch 11: Val Acc = 0.6386
Epoch 12: Val Acc = 0.6425
Epoch 13: Val Acc = 0.6427
Epoch 14: Val Acc = 0.6484
Epoch 15: Val Acc = 0.6443
Epoch 16: Val Acc = 0.6468
Epoch 17: Val Acc = 0.6491
Epoch 18: Val Acc = 0.6471
Epoch 19: Val Acc = 0.6480
Epoch 20: Val Acc = 0.6491
Epoch 21: Val Acc = 0.6452
Epoch 22: Val Acc = 0.6468
Early stopping at epoch 22

✅ TFT Best Val Accuracy: 0.6491


## 🔬 Step 8: Train Stockformer Model

In [9]:
class StockformerModel(nn.Module):
    def __init__(self):
        super().__init__()
        d, h, l = config.STOCKFORMER_D_MODEL, config.STOCKFORMER_N_HEADS, config.STOCKFORMER_N_LAYERS

        self.trend_enc = nn.Sequential(nn.Linear(40, d), nn.LayerNorm(d), nn.GELU())
        self.seasonal_enc = nn.Sequential(nn.Linear(40, d), nn.LayerNorm(d), nn.GELU())
        self.residual_enc = nn.Sequential(nn.Linear(40, d), nn.LayerNorm(d), nn.GELU())

        self.pos = nn.Parameter(torch.randn(1, 60, d) * 0.02)

        enc = nn.TransformerEncoderLayer(d, h, d*4, 0.1, batch_first=True)
        self.trend_tf = nn.TransformerEncoder(enc, l)
        self.seasonal_tf = nn.TransformerEncoder(nn.TransformerEncoderLayer(d, h, d*4, 0.1, batch_first=True), l)
        self.residual_tf = nn.TransformerEncoder(nn.TransformerEncoderLayer(d, h, d*4, 0.1, batch_first=True), l)

        self.fusion = nn.Sequential(nn.Linear(d*3, d*2), nn.GELU(), nn.Linear(d*2, d))
        self.output = nn.Linear(d, 3)

    def forward(self, x):
        b, s, _ = x.shape
        trend = x.cumsum(1) / torch.arange(1, s+1, device=x.device).view(1, -1, 1)
        seasonal = x - trend

        t = self.trend_tf(self.trend_enc(trend) + self.pos[:, :s])[:, -1]
        se = self.seasonal_tf(self.seasonal_enc(seasonal) + self.pos[:, :s])[:, -1]
        r = self.residual_tf(self.residual_enc(x) + self.pos[:, :s])[:, -1]

        return self.output(self.fusion(torch.cat([t, se, r], -1)))

print("="*60)
print("🚀 TRAINING STOCKFORMER MODEL")
print("="*60)

sf_model = StockformerModel().to(DEVICE)
optimizer = torch.optim.AdamW(sf_model.parameters(), lr=config.LEARNING_RATE)

best_acc, patience_count, best_state = 0, 0, None

for epoch in range(config.MAX_EPOCHS):
    # Train
    sf_model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        out = sf_model(batch['features'].to(DEVICE))
        loss = criterion(out, batch['labels'].to(DEVICE))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(sf_model.parameters(), 1.0)
        optimizer.step()

    # Validate
    sf_model.eval()
    correct = sum((sf_model(b['features'].to(DEVICE)).argmax(1) == b['labels'].to(DEVICE)).sum().item()
                  for b in val_loader)
    acc = correct / len(val_dataset)
    print(f"Epoch {epoch+1}: Val Acc = {acc:.4f}")

    if acc > best_acc:
        best_acc, patience_count = acc, 0
        best_state = {k: v.cpu().clone() for k, v in sf_model.state_dict().items()}
    else:
        patience_count += 1
        if patience_count >= config.PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break

sf_model.load_state_dict(best_state)
print(f"\n✅ Stockformer Best Val Accuracy: {best_acc:.4f}")

🚀 TRAINING STOCKFORMER MODEL
Epoch 1: Val Acc = 0.6006
Epoch 2: Val Acc = 0.6085
Epoch 3: Val Acc = 0.6108
Epoch 4: Val Acc = 0.6181
Epoch 5: Val Acc = 0.6080
Epoch 6: Val Acc = 0.6183
Epoch 7: Val Acc = 0.5944
Epoch 8: Val Acc = 0.5990
Epoch 9: Val Acc = 0.6032
Epoch 10: Val Acc = 0.6057
Epoch 11: Val Acc = 0.5935
Early stopping at epoch 11

✅ Stockformer Best Val Accuracy: 0.6183


## 📊 Step 9: Ensemble Evaluation

In [10]:
print("="*60)
print("📊 ENSEMBLE EVALUATION ON TEST SET")
print("="*60)

# Get test data
X_test = test_dataset.features.numpy()
y_test = test_dataset.labels.numpy()

# CatBoost predictions
cat_probs = catboost_model.predict_proba(X_test[:, -1, :])

# TFT predictions
tft_model.eval()
with torch.no_grad():
    tft_probs = torch.softmax(tft_model(torch.FloatTensor(X_test).to(DEVICE)), 1).cpu().numpy()

# Stockformer predictions
sf_model.eval()
with torch.no_grad():
    sf_probs = torch.softmax(sf_model(torch.FloatTensor(X_test).to(DEVICE)), 1).cpu().numpy()

# Ensemble (weighted average)
ensemble_probs = (config.CATBOOST_WEIGHT * cat_probs +
                  config.TFT_WEIGHT * tft_probs +
                  config.STOCKFORMER_WEIGHT * sf_probs)

ensemble_preds = np.argmax(ensemble_probs, axis=1)
ensemble_conf = np.max(ensemble_probs, axis=1)
ensemble_acc = (ensemble_preds == y_test).mean()

print(f"\n🎯 ENSEMBLE TEST ACCURACY: {ensemble_acc:.4f} ({ensemble_acc*100:.1f}%)")

print(f"\n📋 Individual Model Accuracy:")
print(f"   CatBoost:    {(np.argmax(cat_probs, 1) == y_test).mean():.4f}")
print(f"   TFT:         {(np.argmax(tft_probs, 1) == y_test).mean():.4f}")
print(f"   Stockformer: {(np.argmax(sf_probs, 1) == y_test).mean():.4f}")

# Model agreement analysis
cat_preds = np.argmax(cat_probs, axis=1)
tft_preds = np.argmax(tft_probs, axis=1)
sf_preds = np.argmax(sf_probs, axis=1)
all_preds = np.stack([cat_preds, tft_preds, sf_preds], axis=1)
agreements = (all_preds == ensemble_preds[:, None]).sum(axis=1)

print(f"\n🤝 Agreement Analysis:")
for i in [1, 2, 3]:
    mask = agreements == i
    if mask.sum() > 0:
        acc = (ensemble_preds[mask] == y_test[mask]).mean()
        print(f"   {i}/3 agree: {mask.sum():,} samples, accuracy: {acc:.4f}")

print(f"\n📈 Confidence Analysis:")
for thresh in [60, 70, 80]:
    mask = ensemble_conf * 100 >= thresh
    if mask.sum() > 0:
        acc = (ensemble_preds[mask] == y_test[mask]).mean()
        print(f"   Conf >= {thresh}%: {mask.sum():,} ({mask.mean()*100:.1f}%), accuracy: {acc:.4f}")

print("\n" + "="*60)
print(classification_report(y_test, ensemble_preds, target_names=['SHORT', 'NEUTRAL', 'LONG']))

📊 ENSEMBLE EVALUATION ON TEST SET

🎯 ENSEMBLE TEST ACCURACY: 0.5825 (58.2%)

📋 Individual Model Accuracy:
   CatBoost:    0.5701
   TFT:         0.5822
   Stockformer: 0.5607

🤝 Agreement Analysis:
   1/3 agree: 163 samples, accuracy: 0.4601
   2/3 agree: 888 samples, accuracy: 0.5090
   3/3 agree: 2,755 samples, accuracy: 0.6134

📈 Confidence Analysis:
   Conf >= 60%: 1,443 (37.9%), accuracy: 0.6715
   Conf >= 70%: 280 (7.4%), accuracy: 0.8214
   Conf >= 80%: 58 (1.5%), accuracy: 0.9655

              precision    recall  f1-score   support

       SHORT       0.75      0.29      0.41      1300
     NEUTRAL       0.55      0.90      0.69      1879
        LONG       0.61      0.24      0.35       629

    accuracy                           0.58      3808
   macro avg       0.64      0.48      0.48      3808
weighted avg       0.63      0.58      0.54      3808



## 💾 Step 10: Save Models to Google Drive

In [11]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create save directory
os.makedirs(config.MODEL_SAVE_PATH, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

print("💾 Saving models...")

# Save CatBoost
catboost_model.save_model(f"{config.MODEL_SAVE_PATH}catboost_model.cbm")

# Save TFT
torch.save(tft_model.state_dict(), f"{config.MODEL_SAVE_PATH}tft_model.pt")

# Save Stockformer
torch.save(sf_model.state_dict(), f"{config.MODEL_SAVE_PATH}stockformer_model.pt")

# Save config and features
model_config = {
    "feature_columns": FEATURE_NAMES,
    "num_features": 40,
    "lookback_days": config.LOOKBACK_DAYS,
    "ensemble_weights": {
        "catboost": config.CATBOOST_WEIGHT,
        "tft": config.TFT_WEIGHT,
        "stockformer": config.STOCKFORMER_WEIGHT
    },
    "thresholds": {
        "up": config.UP_THRESHOLD,
        "down": config.DOWN_THRESHOLD
    },
    "test_accuracy": float(ensemble_acc),
    "trained_at": timestamp
}

with open(f"{config.MODEL_SAVE_PATH}model_config.json", "w") as f:
    json.dump(model_config, f, indent=2)

print(f"\n✅ All models saved to: {config.MODEL_SAVE_PATH}")
print(f"   📁 catboost_model.cbm")
print(f"   📁 tft_model.pt")
print(f"   📁 stockformer_model.pt")
print(f"   📁 model_config.json")
print(f"\n🎉 Training complete! Test accuracy: {ensemble_acc*100:.1f}%")

Mounted at /content/drive
💾 Saving models...

✅ All models saved to: /content/drive/MyDrive/SwingAI/models/
   📁 catboost_model.cbm
   📁 tft_model.pt
   📁 stockformer_model.pt
   📁 model_config.json

🎉 Training complete! Test accuracy: 58.2%


## 🎉 Training Complete!

Your models are saved to Google Drive. Download them and upload to Modal for production inference.

---
**Next Steps:**
1. Download models from Google Drive
2. Upload to Modal volume using `ml/inference/modal_inference.py`
3. Deploy backend to Railway
4. Test signal generation!